In [135]:
import gurobipy as gb
from gurobipy import *

In [136]:
cycletime = [96,96,48,72,48]

sites = ["Goldsborg", "Saratoga", "Kingston", "Washington","New Bern"]

port_weights=[0.1,0.1,0.4,0.2,0.2]
num_wagons= 40
load_wagon = 75
trains = ["train 1", "train 2", "train 3","train 4","train 5","train 6","train 7","train 8"]
S=len(sites)
T=len(trains)


totaltime = 600  #hours in month


In [137]:
model = gb.Model("Train Utilization")

In [138]:
V = model.addVars(T,S, vtype=GRB.INTEGER, lb=0, name= ['no of visit by '+ train[i] +" to site  "+sites[j] for i in range(T) for j in range(S)])
# Define the total_trip_count variable
total_trip_count = model.addVar(vtype=GRB.INTEGER, name="total_trip_count")


In [139]:
#Objective function maximisation of utilization
model.setObjective(sum(quicksum((V[i,j] for j in range(S))) for i in range(T)), GRB.MAXIMIZE)

In [140]:

# Add constraint for total_trip_count
model.addConstr(total_trip_count == sum(V[i,j] for i in range(T) for j in range(S)))
# Add your original constraint
for i in range(S):
    model.addConstr(sum(V[j,i] for j in range(T)) == port_weights[i] * total_trip_count)

#Adding constraint that cycle time should not exceed 720 hours
#model.addConstr(sum(quicksum((V[i,j]*cycletime[j] for j in range(S))) for i in range(T)) <= totaltime*T)

#Adding the constraint that individual time of each train should not exceed 720 hours
for i in range(T):
    model.addConstr(quicksum((V[i,j]*cycletime[j] for j in range(S)))<= totaltime)


In [141]:
# Adding the constraint to ensure that utilization is between 70% and 76%
# for i in range(T):
#     utilization = sum(V[i,j]*cycletime[j] for j in range(S))/720*100

#     model.addConstr(utilization >= 0.7)
#     model.addConstr(utilization <= 0.76)


In [142]:
model.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 14 rows, 41 columns and 126 nonzeros
Model fingerprint: 0x014d77e7
Variable types: 0 continuous, 41 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+02, 6e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 13 rows, 41 columns, 85 nonzeros
Variable types: 0 continuous, 41 integer (0 binary)

Root relaxation: objective 7.692308e+01, 19 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   76.92308    0   13   -0.00000   76.92308      -     -    0s
H    0     0   

In [146]:
# Print the optimal value of the decision variables

# Print the optimal objective value
print('Obj:', model.objVal)

#Gettign the values for the decision variables
for v in model.getVars():
    if v.x > 0:
        print(v.varName, v.x)


#Printing the total time each train has been used
for i in range(T):
    print("Total time train ", i+1, "has been used is ", sum(V[i,j].x*cycletime[j] for j in range(S)))
#now calculating the utlizing of each train iin percentage terms based on 720 hours avaialable
train_utilization = []
for i in range(T):
    print("Utilization of train ", i+1, "is ", sum(V[i,j].x*cycletime[j] for j in range(S))/720*100, "%")
    train_utilization.append(sum(V[i,j].x*cycletime[j] for j in range(S))/720*100)


#Getting average utilization of all trains
print("Average utilization of all trains is ", sum(train_utilization)/len(train_utilization), "%")

Obj: 70.0
no of visit by train 1 to site  Goldsborg 3.0
no of visit by train 1 to site  Kingston 2.0
no of visit by train 1 to site  New Bern 4.0
no of visit by train 2 to site  Goldsborg 2.0
no of visit by train 2 to site  Kingston 8.0
no of visit by train 3 to site  Kingston 8.0
no of visit by train 3 to site  Washington 3.0
no of visit by train 4 to site  Saratoga 3.0
no of visit by train 4 to site  Kingston 3.0
no of visit by train 4 to site  New Bern 3.0
no of visit by train 5 to site  Goldsborg 2.0
no of visit by train 5 to site  Washington 3.0
no of visit by train 6 to site  Kingston 2.0
no of visit by train 6 to site  Washington 5.0
no of visit by train 6 to site  New Bern 1.0
no of visit by train 7 to site  Saratoga 4.0
no of visit by train 7 to site  Kingston 4.0
no of visit by train 8 to site  Kingston 1.0
no of visit by train 8 to site  Washington 3.0
no of visit by train 8 to site  New Bern 6.0
total_trip_count 70.0
Total time train  1 has been used is  576.0
Total time tr